In [ ]:
#Importing Libraries
import nltk
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger') 
from nltk.tokenize import sent_tokenize, word_tokenize 
import pandas as pd
import os
from nltk.corpus import stopwords
import string
from nltk.stem import PorterStemmer 
from nltk.stem import WordNetLemmatizer 

In [ ]:
#Opening files and appending data to corpus
data = []
file_names = []
for root, _, files in os.walk("stories"):
    for file in files:
        if not file.startswith('.') and not file.endswith('.html'):
            file_names.append(file)
            with open(os.path.join(root, file), 'rb') as f:
                text = f.read()
                data.append(text)
                
corpus = pd.DataFrame(list(zip(file_names, data)), columns = ['file', 'text'])
corpus.head()

,file,text
0,bulprint.txt,b'\r\n Prints of t...
1,grav,b'\r\nMEMORY CEMETERY\r\n by Gay Bost\r\n \r...
2,healer.txt,b' ...
3,blue,b' \r\nSOLDIER BLUE\r\n by Rick A...
4,traitor.txt,b'\r\n TRE...


In [ ]:
#Preprocessing
def preprocess(text):
  #Decoding text
    text = text.decode("latin1")

  #Creating Words
    words = word_tokenize(text)

  #Removing Stop Words and Punctuation
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in words]
    words = [word for word in stripped if word.isalpha()]
    words = [word for word in words if word not in stopwords.words('english')]

  #Lemmatize Words
    wn = WordNetLemmatizer()
    words = [wn.lemmatize(word) for word in words]
    words = [word.lower() for word in words]

    return words

corpus['words'] = corpus.text.apply(lambda x: preprocess(x))

#Verifying task completion
corpus.head()

,file,text,words
0,bulprint.txt,b'\r\n Prints of t...,"[prints, city, g, daniel, flower, detective, g..."
1,grav,b'\r\nMEMORY CEMETERY\r\n by Gay Bost\r\n \r...,"[memory, cemetery, gay, bost, i, nt, like, hal..."
2,healer.txt,b' ...,"[healertxt, ben, blumenberg, reality, software..."
3,blue,b' \r\nSOLDIER BLUE\r\n by Rick A...,"[soldier, blue, rick, arnold, i, ca, nt, conti..."
4,traitor.txt,b'\r\n TRE...,"[treason, marcus, tullius, cicero, bc, roman, ..."


In [ ]:
len(corpus)

467

In [ ]:
#Creating Inverted Index
invIdx = {}
for idx, row in corpus.iterrows():
    for word in row.words:
        if word in invIdx:
            invIdx[word].add(row['file'])
        else:
            invIdx[word] = set()
            invIdx[word].add(row['file'])

for key in invIdx:
    invIdx[key] = list(invIdx[key])
    invIdx[key].sort()

print(invIdx['monkey'])

['7voysinb.txt', 'aesop11.txt', 'breaks3.asc', 'curious.george', 'darkness.txt', 'dwar', 'fearmnky', 'gulliver.txt', 'hitch2.txt', 'hop-frog.poe', 'hound-b.txt', 'lionwar.txt', 'luf', 'mario.txt', 'monkking.txt', 'monksol.txt', 'rocket.sf', 'tearglas.txt', 'timem.hac', 'wisteria.txt']


In [ ]:
#Query Definition
def AND(docs_a, docs_b):
 
    docs = []
    comparisons = 0
    while (len(docs_a) > 0 and len(docs_b) > 0):
        if docs_a[0] > docs_b[0]:
            comparisons = comparisons + 1
            docs_b = docs_b[1:]
        elif docs_a[0] < docs_b[0]:
            comparisons = comparisons + 1
            docs_a = docs_a[1:]
        else:
            comparisons = comparisons + 1
            docs.append(docs_a[0])
            docs_a = docs_a[1:]
            docs_b = docs_b[1:]
#     print(len(docs))
    return docs, comparisons

def OR(docs_a, docs_b):
  
    docs = []
    comparisons = 0

    while (len(docs_a) > 0 or len(docs_b) > 0):
        if (len(docs_a) > 0 and len(docs_b) > 0 and docs_a[0] > docs_b[0]) or len(docs_a) == 0:
            comparisons = comparisons + 1
            docs.append(docs_b[0])
            docs_b = docs_b[1:]
        elif (len(docs_a) > 0 and len(docs_b) > 0 and docs_a[0] < docs_b[0]) or len(docs_b) == 0:
            comparisons = comparisons + 1
            docs.append(docs_a[0])
            docs_a = docs_a[1:]
        else:
            comparisons = comparisons + 1
            docs.append(docs_a[0])
            docs_a = docs_a[1:]
            docs_b = docs_b[1:]
    
    return docs, comparisons

#         print(a,"OR",b,":", len(docs), "->", docs)
#         print("Number of Comparisons:", comparisons)
        

def AND_NOT(docs_a, docs_b):
    temp, comparisons = AND(docs_a.copy(), docs_b.copy())
#     print(len(temp))
    docs = []
    while len(docs_a) > 0:
        if len(temp)==0:
            docs.append(docs_a)
            docs_a = docs_a[1:]
        elif docs_a[0] < temp[0] or docs_a[0]>temp[0]:
            docs.append(docs_a[0])
            docs_a = docs_a[1:]
            comparisons += 1
        elif docs_a[0] == temp[0]:
            docs_a = docs_a[1:]
            temp = temp[1:]
            comparisons += 1
    return docs, comparisons
    
#     print(a,"AND NOT",b,":", len(docs), "->", docs)
#     print()

def OR_NOT(docs_a, docs_containing_b):
    not_comparisons = 0
    docs_not_b = []

  # comparisons for docs NOT b
    for a in corpus['file']:
        if a not in docs_containing_b:
            not_comparisons += 1
            docs_not_b.append(a)

    docs_not_b.sort()
    # ORing docs_a and docs_not_b
    temp, c_temp = OR(docs_a.copy(), docs_not_b.copy())

    return temp, not_comparisons+c_temp

In [ ]:
# corpus.to_csv("preprocessed_corpus.csv", index=False)

In [ ]:
len(invIdx["roads"])

4

In [ ]:
#Main Executer Function
def main():
    query = input('Query: ')
    operations = input('Operations: ')
    query_words = preprocess(query.encode(encoding='latin1'))
    ops = operations.split(",")
    
    tc = 0
#     print(query_words)
#     print(len(ops))
    if len(query_words) != len(ops)+1:
        print("Invalid input")
        return
    else:
        for i, op in enumerate(ops):
            if i==0:
                if op == "AND":
                    docs, comparisons = AND(invIdx[query_words[i]].copy(),invIdx[query_words[i+1]].copy())
                elif op == "OR":
                    docs, comparisons = OR(invIdx[query_words[i]].copy(),invIdx[query_words[i+1]].copy())
                elif op == "AND NOT":
                    docs, comparisons = AND_NOT(invIdx[query_words[i]].copy(),invIdx[query_words[i+1]].copy())
                elif op == "OR NOT":
                    docs, comparisons = OR_NOT(invIdx[query_words[i]].copy(),invIdx[query_words[i+1]].copy())
                else:
                    print("Invalid Query")
                    return
            else:
                if op == "AND":
                    docs, comparisons = AND(docs.copy(),invIdx[query_words[i+1]].copy())
                elif op == "OR":
                    docs, comparisons = OR(docs.copy(),invIdx[query_words[i+1]].copy())
                elif op == "AND NOT":
                    docs, comparisons = AND_NOT(docs.copy(),invIdx[query_words[i+1]].copy())
                elif op == "OR NOT":
                    docs, comparisons = OR_NOT(docs.copy(),invIdx[query_words[i+1]].copy())
                else:
                    print("Invalid Query")
                    return
            tc += comparisons
        print("Total Docs:{} ".format(len(docs)))
        print("Total Comparisons:", tc)
        print("Docs:{}".format(docs))

if __name__ == "__main__":
    main()

Query: telephone, paved, roadses
Operations: OR NOT,AND NOT
Total Docs:459 
Total Comparisons: 1176
Docs:['100west.txt', '13chil.txt', '14.lws', '16.lws', '17.lws', '18.lws', '19.lws', '20.lws', '3gables.txt', '3lpigs.txt', '3sonnets.vrs', '3student.txt', '3wishes.txt', '4moons.txt', '5orange.txt', '6ablemen.txt', '6napolen.txt', '7oldsamr.txt', '7voysinb.txt', 'ab40thv.txt', 'abbey.txt', 'adler.txt', 'adv_alad.txt', 'advsayed.txt', 'advtthum.txt', 'aesop11.txt', 'aesopa10.txt', 'aircon.txt', 'aisle.six', 'aislesix.txt', 'alad10.txt', 'alissadl.txt', 'altside.hum', 'aluminum.hum', 'aminegg.txt', 'angelfur.hum', 'angry_ca.txt', 'antcrick.txt', 'aquith.txt', 'arcadia.sty', 'archive', 'arctic.txt', 'asop', 'bagel.man', 'bagelman.txt', 'batlslau.txt', 'beast.asc', 'beautbst.txt', 'beggars.txt', 'bern', 'berternie.txt', 'bestwish', 'beyond.hum', 'bgb.txt', 'bgcspoof.txt', 'bigred.hum', 'bishop00.txt', 'blabnove.hum', 'blabnove.txt', 'blackp.txt', 'blackrdr', 'blak', 'blasters.fic', 'blh.txt

In [ ]:
len(OR(invIdx["lion"].copy(), invIdx["stood"].copy())[0])

210

In [ ]:
len(invIdx["lion"])

23

In [ ]:
len(invIdx["stood"])

202

In [ ]:
count = 0
for f in invIdx["lion"]:
    if f in invIdx["stood"]:
        count += 1
count

15

In [ ]:
docs = ['100west.txt', '13chil.txt', '14.lws', '16.lws', '17.lws', '18.lws', '19.lws', '20.lws', '3gables.txt', '3lpigs.txt', '3sonnets.vrs', '3student.txt', '3wishes.txt', '4moons.txt', '5orange.txt', '6ablemen.txt', '6napolen.txt', '7oldsamr.txt', '7voysinb.txt', 'ab40thv.txt', 'abbey.txt', 'abyss.txt', 'adler.txt', 'adv_alad.txt', 'advsayed.txt', 'advtthum.txt', 'aesop11.txt', 'aesopa10.txt', 'aircon.txt', 'aisle.six', 'aislesix.txt', 'alad10.txt', 'alissadl.txt', 'altside.hum', 'aluminum.hum', 'aminegg.txt', 'angelfur.hum', 'angry_ca.txt', 'antcrick.txt', 'aquith.txt', 'arcadia.sty', 'archive', 'arctic.txt', 'asop', 'assorted.txt', 'bagel.man', 'bagelman.txt', 'batlslau.txt', 'beast.asc', 'beautbst.txt', 'beggars.txt', 'bern', 'berternie.txt', 'bestwish', 'beyond.hum', 'bgb.txt', 'bgcspoof.txt', 'bigred.hum', 'bishop00.txt', 'blabnove.hum', 'blabnove.txt', 'blackp.txt', 'blackrdr', 'blak', 'blasters.fic', 'blh.txt', 'blind.txt', 'blossom.pom', 'blue', 'bluebrd.txt', 'bookem.1', 'bookem2', 'bookem3', 'brain.damage', 'bram', 'bran', 'breaks1.asc', 'breaks2.asc', 'breaks3.asc', 'bruce-p.txt', 'buggy.txt', 'buldetal.txt', 'buldream.txt', 'bulfelis.txt', 'bulhuntr.txt', 'bulironb.txt', 'bullove.txt', 'bulmrx.txt', 'bulnland.txt', 'bulnoopt.txt', 'bulolli1.txt', 'bulolli2.txt', 'bulphrek.txt', 'bulprint.txt', 'bulwer.lytton', 'bulzork1.txt', 'bumm', 'bureau.txt', 'burintrv.66', 'burintrv.78', 'burintrv.92', 'burltrs', 'burn', 'cabin.txt', 'cameloto.hum', 'campfire.txt', 'candle.hum', 'cardcnt.txt', 'ccm.txt', 'charlie.txt', 'chik', 'clevdonk.txt', 'clon', 'cmoutmou.txt', 'comp', 'confilct.fun', 'consumdr.hum', 'contrad1.hum', 'cooldark.sto', 'cooldark.txt', 'corcor.hum', 'cow.exploder', 'crabhern.txt', 'crazy.hum', 'cum', 'curious.george', 'cybersla.txt', 'dakota.txt', 'dan', 'darkness.txt', 'day.in.mcdonald', 'deal', 'deathmrs.d', 'deer.txt', 'descent.poe', 'diaryflf.txt', 'dicegame.txt', 'dicksong.txt', 'disco.be.fun', 'discocanbefun.txt', 'domain.poe', 'dopedenn.txt', 'dskool.txt', 'dtruck.txt', 'dwar', 'elite.app', 'elveshoe.txt', 'emperor3.txt', 'empnclot.txt', 'empsjowk.txt', 'empty.txt', 'enc', 'encamp01.txt', 'enginer.txt', 'enya_trn.txt', 'excerpt.hum', 'excerpt.txt', 'eyeargon.hum', 'ezoff', 'fable.txt', 'fantas.hum', 'fantasy.hum', 'fantasy.txt', 'fea1', 'fea2', 'fea3', 'fear.hum', 'fearmnky', 'fgoose.txt', 'fic1', 'fic2', 'fic3', 'fic4', 'fic5', 'fic7', 'fish.txt', 'fleas.txt', 'flktrp.txt', 'floobs.txt', 'flute.txt', 'flytrunk.txt', 'forgotte', 'fourth.fic', 'fowl.death', 'foxncrow.txt', 'foxngrap.txt', 'foxnstrk.txt', 'fran', 'fred.txt', 'freeman.fil', 'friend.s', 'friends.txt', 'frogp.txt', 'frum', 'game.txt', 'gatherng.txt', 'gay', 'gemdra.txt', 'ghost', 'girl', 'girlclub.txt', 'glimpse1.txt', 'gloves.txt', 'gold3ber.txt', 'goldbug.poe', 'goldenp.txt', 'goldfish.txt', 'goldgoos.txt', 'grav', 'graymare.txt', 'greatlrn.leg', 'greedog.txt', 'gulliver.txt', 'hansgrtl.txt', 'hareleph.txt', 'hareporc.txt', 'haretort.txt', 'healer.txt', 'hell4.txt', 'hellmach.txt', 'helmfuse.txt', 'hils', 'hitch2.txt', 'hitch3.txt', 'hole2nar.txt', 'holmesbk.txt', 'home.fil', 'hop-frog.poe', 'horsdonk.txt', 'horswolf.txt', 'hotline1.txt', 'hotline3.txt', 'hotline4.txt', 'hound-b.txt', 'how.ernie.bert', 'idi.hum', 'igiv', 'imagin.hum', 'immortal', 'immorti.hum', 'imonly17.txt', 'inter', 'island.poe', 'jackbstl.txt', 'jackmac.fic', 'jaynejob.asc', 'jerichms.hum', 'jim.asc', 'keeping.insanit', 'keepmodu.txt', 'kharian.txt', 'kneeslapper', 'kneeslapper.txt', 'knuckle.txt', 'korea.s', 'kzap.txt', 'ladylust.hum', 'lament.txt', 'lgoldbrd.txt', 'life.txt', 'lil', 'lionbird', 'lionmane.txt', 'lionmosq.txt', 'lionwar.txt', 'lmermaid.txt', 'lmtchgrl.txt', 'long1-3.txt', 'lpeargrl.txt', 'lrrhood.txt', 'ltp', 'luf', 'lure.txt', 'mario.txt', 'mattress.txt', 'mazarin.txt', 'mcdonaldl.txt', 'melissa.txt', 'mike.txt', 'mindprob.txt', 'mindwar', 'missing.txt', 'modemhippy.txt', 'monkking.txt', 'monksol.txt', 'mouslion.txt', 'mtinder.txt', 'musibrem.txt', 'mydream.txt', 'myeyes', 'narciss.txt', 'nigel.1', 'nigel.10', 'nigel.2', 'nigel.3', 'nigel.4', 'nigel.5', 'nigel.6', 'nigel.7', 'nihgel_8.9', 'nitepeek.sto', 'non2', 'non3', 'non4', 'obstgoat.txt', 'omarsheh.txt', 'outcast.dos', 'oxfrog.txt', 'paink-ws.txt', 'panama.txt', 'parotsha.txt', 'partya.txt', 'paul_har.sto', 'peace.fun', 'pepdegener.txt', 'pepsi.degenerat', 'perf', 'pinocch.txt', 'piracy.sto', 'plescopm.txt', 'poem-1.txt', 'poem-2.txt', 'poem-4.txt', 'poplstrm.txt', 'pphamlin.txt', 'pregn.txt', 'prince.art', 'progx', 'psf.txt', 'psi', 'psyc', 'pussboot.txt', 'qcarroll', 'quarter.c1', 'quarter.c10', 'quarter.c11', 'quarter.c12', 'quarter.c13', 'quarter.c14', 'quarter.c15', 'quarter.c16', 'quarter.c17', 'quarter.c18', 'quarter.c19', 'quarter.c2', 'quarter.c3', 'quarter.c4', 'quarter.c5', 'quarter.c6', 'quarter.c7', 'quarter.c8', 'quarter.c9', 'quest', 'quickfix', 'quot', 'radar_ra.txt', 'rainda.txt', 'reality.txt', 'reap', 'redragon.txt', 'retrib.txt', 'rid.txt', 'robotech', 'rock', 'rocket.sf', 'roger1.txt', 'running.txt', 's&m_plot', 's&m_that', 'safe', 'sanpedr2.txt', 'shoscomb.txt', 'shrdfarm.txt', 'shulk.txt', 'sick-kid.txt', 'sight.txt', 'silverb.txt', 'sis', 'sleprncs.txt', 'snow.txt', 'snowmaid.txt', 'snowqn1.txt', 'social.vikings', 'socialvikings.txt', 'solitary.txt', 'space.txt', 'spam.key', 'spectacl.poe', 'spider.txt', 'spiders.txt', 'sqzply.txt', 'sre-dark.txt', 'sre01.txt', 'sre02.txt', 'sre03.txt', 'sre04.txt', 'sre05.txt', 'sre06.txt', 'sre07.txt', 'sre08.txt', 'sre09.txt', 'sre10.txt', 'sre_feqh.txt', 'sre_finl.txt', 'sre_sei.txt', 'sretrade.txt', 'srex.txt', 'stainles.ana', 'stairdre.txt', 'startrek.txt', 'stsgreek', 'sucker.txt', 'sunday.txt', 'superg1', 'szechuan', 't_zone.jok', 'tailbear.txt', 'tao3.dos', 'taxnovel.txt', 'tcoa.txt', 'tctac.txt', 'tearglas.txt', 'telefone.txt', 'terrorbears.txt', 'testpilo.hum', 'textfile.primer', 'thanksg', 'the-tree.txt', 'thewave', 'timem.hac', 'times.fic', 'timetrav.txt', 'tin', 'tinsoldr.txt', 'toilet.s', 'traitor.txt', 'tree.txt', 'tuc_mees', 'uglyduck.txt', 'unluckwr.txt', 'vaincrow.txt', 'vainsong.txt', 'valen', 'vampword.txt', 'vday.hum', 'veiledl.txt', 'vgilante.txt', 'wall.art', 'wanderer.fun', 'weaver.txt', 'weeprncs.txt', 'whgdsreg.reg', 'wisteria.txt', 'withdraw.cyb', 'wlgirl.txt', 'wolf7kid.txt', 'wolfcran.txt', 'wolflamb.txt', 'wombat.und', 'write', 'wrt', 'yukon.txt', 'zombies.txt']

In [ ]:
count = 0
for d in docs:
    if d in invIdx["road"]:
        count+=1
count

116

In [ ]:
invIdx["roads"]

['abyss.txt', 'assorted.txt', 'breaks3.asc', 'cybersla.txt']

In [ ]:
AND_NOT(docs, invIdx["roads"])

4


(['100west.txt',
  '13chil.txt',
  '14.lws',
  '16.lws',
  '17.lws',
  '18.lws',
  '19.lws',
  '20.lws',
  '3gables.txt',
  '3lpigs.txt',
  '3sonnets.vrs',
  '3student.txt',
  '3wishes.txt',
  '4moons.txt',
  '5orange.txt',
  '6ablemen.txt',
  '6napolen.txt',
  '7oldsamr.txt',
  '7voysinb.txt',
  'ab40thv.txt',
  'abbey.txt',
  'adler.txt',
  'adv_alad.txt',
  'advsayed.txt',
  'advtthum.txt',
  'aesop11.txt',
  'aesopa10.txt',
  'aircon.txt',
  'aisle.six',
  'aislesix.txt',
  'alad10.txt',
  'alissadl.txt',
  'altside.hum',
  'aluminum.hum',
  'aminegg.txt',
  'angelfur.hum',
  'angry_ca.txt',
  'antcrick.txt',
  'aquith.txt',
  'arcadia.sty',
  'archive',
  'arctic.txt',
  'asop',
  'bagel.man',
  'bagelman.txt',
  'batlslau.txt',
  'beast.asc',
  'beautbst.txt',
  'beggars.txt',
  'bern',
  'berternie.txt',
  'bestwish',
  'beyond.hum',
  'bgb.txt',
  'bgcspoof.txt',
  'bigred.hum',
  'bishop00.txt',
  'blabnove.hum',
  'blabnove.txt',
  'blackp.txt',
  'blackrdr',
  'blak',
  'bla